<a href="https://colab.research.google.com/github/CeliaDiazSanchez/Data-Mining/blob/LAB01/LAB01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
import requests
import pandas as pd

# URL del archivo CSV en GitHub
url = "https://raw.githubusercontent.com/CeliaDiazSanchez/Data-Mining/LAB01/vacunaciones.csv"

# Realizar la solicitud GET para obtener el contenido del archivo
response = requests.get(url)

# Verificar si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Guardar el contenido del archivo en un archivo local
    with open('vacunaciones.csv', 'wb') as f:
        f.write(response.content)
    print("Archivo descargado exitosamente.")
else:
    print("Error al descargar el archivo.")

# Cargar el archivo CSV en un DataFrame de pandas
df = pd.read_csv('vacunaciones.csv')

# Ahora puedes trabajar con el DataFrame df como desees
df = df.sort_values(by='date')
df.head(10)

Archivo descargado exitosamente.


,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
880,England,NaN,2020-12-13,55389.0,55389.0,NaN,NaN,NaN,0.10,0.10,NaN,NaN,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
2498,Scotland,NaN,2020-12-13,18996.0,18996.0,NaN,NaN,NaN,0.35,0.35,NaN,NaN,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
3023,United Kingdom,GBR,2020-12-13,86215.0,86215.0,NaN,NaN,NaN,0.13,0.13,NaN,NaN,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
3144,Wales,NaN,2020-12-13,8207.0,8207.0,NaN,NaN,NaN,0.26,0.26,NaN,NaN,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
2005,Northern Ireland,NaN,2020-12-13,3623.0,3623.0,NaN,NaN,NaN,0.19,0.19,NaN,NaN,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
2499,Scotland,NaN,2020-12-14,NaN,NaN,NaN,NaN,5873.0,NaN,NaN,NaN,1075.0,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
881,England,NaN,2020-12-14,NaN,NaN,NaN,NaN,74067.0,NaN,NaN,NaN,1316.0,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
3145,Wales,NaN,2020-12-14,NaN,NaN,NaN,NaN,2223.0,NaN,NaN,NaN,705.0,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
2006,Northern Ireland,NaN,2020-12-14,NaN,NaN,NaN,NaN,1778.0,NaN,NaN,NaN,939.0,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
3024,United Kingdom,GBR,2020-12-14,NaN,NaN,NaN,NaN,83940.0,NaN,NaN,NaN,1236.0,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...


In [69]:

# Ordenar el DataFrame por fecha
df = df.sort_values(by='date')

# Iterar a través del DataFrame para rellenar los valores NaN en la columna 'people_vaccinated'
for index, row in df.iterrows():
    if pd.isna(row['people_vaccinated']):
        date = row['date']
        country = row['country']

        # Obtener todos los valores de los días anteriores (o menos si no existen)
        previous_dates = df[(df['country'] == country) & (df['date'] < date) & (df['people_vaccinated'].notna())]['people_vaccinated']

        # Calcular la media de todos los valores anteriores y rellenar el NaN con ese valor
        if not previous_dates.empty:
            average_value = previous_dates.mean()
            df.at[index, 'people_vaccinated'] = average_value

# Mostrar el DataFrame resultante
df.head(30)

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
880,England,NaN,2020-12-13,55389.0,55389.000000,NaN,NaN,NaN,0.10,0.10,NaN,NaN,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
2498,Scotland,NaN,2020-12-13,18996.0,18996.000000,NaN,NaN,NaN,0.35,0.35,NaN,NaN,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
3023,United Kingdom,GBR,2020-12-13,86215.0,86215.000000,NaN,NaN,NaN,0.13,0.13,NaN,NaN,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
3144,Wales,NaN,2020-12-13,8207.0,8207.000000,NaN,NaN,NaN,0.26,0.26,NaN,NaN,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
2005,Northern Ireland,NaN,2020-12-13,3623.0,3623.000000,NaN,NaN,NaN,0.19,0.19,NaN,NaN,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
2006,Northern Ireland,NaN,2020-12-14,NaN,32681.666667,NaN,NaN,1778.0,NaN,NaN,NaN,939.0,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
3145,Wales,NaN,2020-12-14,NaN,32681.666667,NaN,NaN,2223.0,NaN,NaN,NaN,705.0,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
3024,United Kingdom,GBR,2020-12-14,NaN,32681.666667,NaN,NaN,83940.0,NaN,NaN,NaN,1236.0,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
2499,Scotland,NaN,2020-12-14,NaN,32681.666667,NaN,NaN,5873.0,NaN,NaN,NaN,1075.0,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
881,England,NaN,2020-12-14,NaN,32681.666667,NaN,NaN,74067.0,NaN,NaN,NaN,1316.0,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
